# Многопоточность в Python. Библиотека threading

In [1]:
import sqlite3
import pandas as pd
import random
import time
import threading
from typing import List, Set, Dict

In [2]:
conn = sqlite3.connect("/content/my_data.db", check_same_thread = False)

In [3]:
conn.execute("""DROP TABLE IF EXISTS sales;""")
conn.execute("""CREATE TABLE sales (id INTEGER, product TEXT, val INTEGER);""")

In [4]:
conn.close()

In [5]:
N_CHUNK = 3
LEN_LIST = 1000000
dict_chunk = {}

def generate_dict_chunk(n_chunk:int= N_CHUNK, len_list:int=LEN_LIST) -> dict:
  for n in range(n_chunk):
    list_id = [_ for _ in range(len_list)]
    list_product = ["".join(["pr",str(n)]) for _ in range(len_list)]
    list_val = [random.randint(1,10) for _ in range(len_list)]
    dict_chunk[n] = [(id, product, val) for id, product, val in zip(list_id, list_product, list_val)]
  return dict_chunk

In [6]:
def insert_records(data_chunk:List[Set])->None:
  conn = sqlite3.connect("/content/my_data.db", check_same_thread = False)
  conn.executemany("""INSERT INTO sales(id,product,val) VALUES (?,?,?)""", data_chunk)
  conn.commit()
  conn.close()

In [7]:
def run_threads():
    threads = []
    data = generate_dict_chunk()

    for k, v in data.items():
      threads.append(threading.Thread(target=insert_records, args=(v,)))

    for thread in threads:
        thread.start()
    for thread in threads:
        thread.join()

In [8]:
start = time.perf_counter()
run_threads()
finish = time.perf_counter()
print("Finished in {} seconds".format(finish-start))

Finished in 12.89777911500002 seconds


# Проверка

In [9]:
def select_sqlite(sql, path_database:str="/content/my_data.db")->pd.DataFrame:
    conn = sqlite3.connect(path_database)
    return pd.read_sql(sql, conn)

In [10]:
sql = """
    select * from pragma_compile_options
    where compile_options like 'THREADSAFE=%'
"""

In [11]:
select_sqlite(sql)

,compile_options
0,THREADSAFE=1


In [12]:
sql = """select s.product, count(*) as total_amount from sales as s group by s.product"""

In [13]:
select_sqlite(sql)

,product,total_amount
0,pr0,1000000
1,pr1,1000000
2,pr2,1000000
